In [1]:
# Import Spark SQL and Spark ML libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.regression import DecisionTreeRegressor


In [2]:
csv =sqlContext.sql("select * from salary2")
maxval = sqlContext.sql("select count(PensionContributions) from salary2")
maxval.show()

In [3]:
csv =sqlContext.sql("select * from salary2")
maxval_nine = sqlContext.sql("select count(PensionContributions)*.90 from salary2")
maxval_twen = sqlContext.sql("select count(PensionContributions)*.20 from salary2")
maxval_nine.show()
maxval_twen.show()

In [4]:
rownumber = sqlContext.sql("select ROW_NUMBER() over (ORDER BY PensionContributions) AS Row, PensionContributions from salary2")
rownumber.createOrReplaceTempView("res1")
val_nineper = sqlContext.sql("select PensionContributions,Row from res1 where Row = '277110' ")
val_twenper = sqlContext.sql("select PensionContributions,Row from res1 where Row = '61580' ")
val_twenper.show()
val_nineper.show()

In [5]:
rownumber = sqlContext.sql("select ROW_NUMBER() over (ORDER BY MedicalDentalVision) AS Row, MedicalDentalVision from salary2")
rownumber.createOrReplaceTempView("res1")
val_nineper = sqlContext.sql("select MedicalDentalVision,Row from res1 where Row = '277110' ")
val_twenper = sqlContext.sql("select MedicalDentalVision,Row from res1 where Row = '61580' ")
val_twenper.show()
val_nineper.show()

In [6]:
rownumber = sqlContext.sql("select ROW_NUMBER() over (ORDER BY TotalCompensation) AS Row, TotalCompensation from salary2")
rownumber.createOrReplaceTempView("res1")
val_nineper = sqlContext.sql("select TotalCompensation,Row from res1 where Row = '277110' ")
val_twenper = sqlContext.sql("select TotalCompensation,Row from res1 where Row = '61580' ")
val_twenper.show()
val_nineper.show()

In [7]:
rownumber = sqlContext.sql("select ROW_NUMBER() over (ORDER BY LTDLifeMedicalTax) AS Row, LTDLifeMedicalTax from salary2")
rownumber.createOrReplaceTempView("res1")
val_nineper = sqlContext.sql("select LTDLifeMedicalTax,Row from res1 where Row = '277110' ")
val_twenper = sqlContext.sql("select LTDLifeMedicalTax,Row from res1 where Row = '61580' ")
val_twenper.show()
val_nineper.show()

In [8]:
csv1 = sqlContext.sql(" select cast(PensionContributions as double),cast(MedicalDentalVision as double),cast(TotalCompensation as double),cast(LTDLifeMedicalTax as double) from salary2")
csv1= csv1.dropna()
data = csv1.select("PensionContributions","MedicalDentalVision","LTDLifeMedicalTax", col("TotalCompensation").alias("label")).where(col("PensionContributions") >= ((6321.92))).where (col("PensionContributions") <= (24482.66)).where(col("MedicalDentalVision") >= (6019.92)).where (col("MedicalDentalVision") <= (18807.05)).where(col("TotalCompensation") >= (57087.26)).where (col("TotalCompensation") <= (177859.19)).where (col("LTDLifeMedicalTax") >= (540.22)).where (col("LTDLifeMedicalTax") <= (2303.66))

# Split the data
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

In [9]:
assembler = VectorAssembler(inputCols = ["PensionContributions","MedicalDentalVision","LTDLifeMedicalTax"], outputCol="features")
dt = DecisionTreeRegressor(labelCol ="label",featuresCol="features")

In [10]:
paramGrid = ParamGridBuilder()\
  .addGrid(dt.maxDepth, [5, 10])\
  .build()
# We define an evaluation metric.  This tells CrossValidator how well we are doing by comparing the true labels with predictions.
evaluator = RegressionEvaluator(metricName="rmse", labelCol=dt.getLabelCol(), predictionCol=dt.getPredictionCol())
# Declare the CrossValidator, which runs model tuning for us.
cv = CrossValidator(estimator=dt, evaluator=evaluator, estimatorParamMaps=paramGrid)

In [11]:
pipeline = Pipeline(stages=[assembler, cv])
pipelineModel = pipeline.fit(train)

In [12]:
predictions = pipelineModel.transform(test)

In [13]:
predicted = predictions.select("features", "prediction", "trueLabel")
display(predicted)

In [14]:
evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print "Root Mean Square Error (RMSE) for Decession Tree Model:", rmse

In [15]:
assembler = VectorAssembler(inputCols = ["PensionContributions","MedicalDentalVision","LTDLifeMedicalTax"], outputCol="features")
gbt = GBTRegressor(labelCol="label")

In [16]:
paramGrid = ParamGridBuilder()\
  .addGrid(gbt.maxDepth, [2, 5])\
  .addGrid(gbt.maxIter, [10, 100])\
  .build()
# We define an evaluation metric.  This tells CrossValidator how well we are doing by comparing the true labels with predictions.
evaluator = RegressionEvaluator(metricName="rmse", labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol())
# Declare the CrossValidator, which runs model tuning for us.
cv = CrossValidator(estimator=gbt, evaluator=evaluator, estimatorParamMaps=paramGrid)

In [17]:
pipeline = Pipeline(stages=[assembler, cv])
pipelineModel = pipeline.fit(train)

In [18]:
predictions = pipelineModel.transform(test)

In [19]:
predicted = predictions.select("features", "prediction", "trueLabel")
display(predicted)

In [20]:
evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print "Root Mean Square Error (RMSE) for GBT Regression :", rmse